# [Object Relational Tutorial](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html)

The SQLAlchemy Object Relational Mapper presents a method of associating user-defined Python classes with database tables, and instances of those classes (objects) with rows in their corresponding tables.  
It includes a system that transparently synchronizes all changes in state between objects and their related rows, called a unit of work, as well as a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.  
The ORM is in contrast to the SQLAlchemy Expression Language, upon which the ORM is constructed.  
Whereas the SQL Expression Language, introduced in SQL Expression Language Tutorial, presents a system of representing the primitive constructs of the relational database directly without opinion, the ORM presents a high level and abstracted pattern of usage, which itself is an example of applied usage of the Expression Language.  
While there is overlap among the usage patterns of the ORM and the Expression Language, the similarities are more superficial than they may at first appear.  
One approaches the structure and content of data from the perspective of a user-defined domain model which is transparently persisted and refreshed from its underlying storage model.  
The other approaches it from the perspective of literal schema and SQL expression representations which are explicitly composed into messages consumed individually by the database.  
A successful application may be constructed using the Object Relational Mapper exclusively.  
In advanced situations, an application constructed with the ORM may make occasional usage of the Expression Language directly in certain areas where specific database interactions are required.

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.2.10'

For this tutorial we will use an in-memory-only SQLite database.  
To connect we can use `create_engine()`:

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True); engine.__doc__

'\n    Connects a :class:`~sqlalchemy.pool.Pool` and\n    :class:`~sqlalchemy.engine.interfaces.Dialect` together to provide a\n    source of database connectivity and behavior.\n\n    An :class:`.Engine` object is instantiated publicly using the\n    :func:`~sqlalchemy.create_engine` function.\n\n    See also:\n\n    :doc:`/core/engines`\n\n    :ref:`connections_toplevel`\n\n    '

In [3]:
engine

Engine(sqlite:///:memory:)

The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module.  
With it enabled, we’ll see all the generated SQL produced.  
If you are working through this tutorial and want less output generated, set it to False.  
This tutorial will format the SQL behind a popup window so it doesn’t get in our way; just click the “SQL” links to see what’s being generated.  
The return value of `create_engine()` is an instance of Engine, and it represents the core interface to the database, adapted through a dialect that handles the details of the database and DBAPI in use.  
In this case the SQLite dialect will interpret instructions to the Python built-in sqlite3 module.  
The first time a method like `Engine.execute()` or `Engine.connect()` is called, the Engine establishes a real DBAPI connection to the database, which is then used to emit the SQL.  
When using the ORM, we typically don’t use the Engine directly once created; instead, it’s used behind the scenes by the ORM as we’ll see shortly.  
**Note about Lazy Connecting:**  
The Engine, when first returned by `create_engine()`, has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database.

## [Declare a Mapping](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#declare-a-mapping)

When using the ORM, the configurational process starts by describing the database tables we’ll be dealing with, and then by defining our own classes which will be mapped to those tables.  
In modern SQLAlchemy, these two tasks are usually performed together, using a system known as Declarative, which allows us to create classes that include directives to describe the actual database table that they will be mapped to.  
Classes mapped using the Declarative system are defined in terms of a base class which maintains a catalog of classes and tables relative to that base - this is known as the declarative base class.  
Our application will usually have just one instance of this base in a commonly imported module.  
We create the base class using the `declarative_base()` function, as follows:

In [4]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base(); Base.__doc__

'The most base type'

In [5]:
Base

sqlalchemy.ext.declarative.api.Base

Now that we have a base of the most base type, we can define any number of mapped classes in terms of it.  
We will start with just a single table called **users**, which will store records for the end-users using our application.  
A new class called User will be the class to which we map this table.  
Within the class, we define details about the table to which we’ll be mapping, primarily the table name, and names and datatypes of columns.  
Although the User class defines a `__repr__()` method, it is optional; we only implement it in this tutorial so that our examples show nicely formatted User objects.

In [6]:
from sqlalchemy import Column, Integer, String


class User(Base):
    
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

A class using Declarative at a minimum needs a `__tablename__` attribute, and at least one Column which is part of a primary key.  
SQLAlchemy never makes any assumptions by itself about the table to which a class refers, including that it has no built-in conventions for names, datatypes, or constraints.  
But this doesn’t mean boilerplate is required; instead, you’re encouraged to create your own automated conventions using helper functions and mixin classes, which is described in detail at Mixin and Custom Base Classes.  
When our class is constructed, Declarative replaces all the Column objects with special Python accessors known as descriptors; this is a process known as instrumentation.  
The "instrumented" mapped class will provide us with the means to refer to our table in a SQL context as well as to persist and load the values of columns from the database.  
Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

## [Create a Schema](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#create-a-schema)

With our User class constructed via the Declarative system, we have defined information about our table, known as table metadata.  
The object used by SQLAlchemy to represent this information for a specific table is called the Table object, and here Declarative has made one for us.  
We can see this object by inspecting the `__table__` attribute:

When we declared our class, Declarative used a Python metaclass in order to perform additional activities once the class declaration was complete; within this phase, it then created a Table object according to our specifications, and associated it with the class by constructing a Mapper object.  
This object is a behind-the-scenes object we normally don’t need to deal with directly (though it can provide plenty of information about our mapping when we need it).  

The Table object is a member of a larger collection known as MetaData.  
When using Declarative, this object is available using the `.metadata` attribute of our declarative base class.  

The MetaData is a registry which includes the ability to emit a limited set of schema generation commands to the database.  
As our SQLite database does not actually have a users table present, we can use MetaData to issue `CREATE TABLE` statements to the database for all tables that don’t yet exist.  
Below, we call the `MetaData.create_all()` method, passing in our Engine as a source of database connectivity.  
We will see that special commands are first emitted to check for the presence of the users table, and following that the actual `CREATE TABLE` statement:

In [7]:
Base.metadata.create_all(engine)

2018-08-18 01:20:10,414 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-18 01:20:10,416 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 01:20:10,418 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-18 01:20:10,419 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 01:20:10,421 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-08-18 01:20:10,423 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 01:20:10,425 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2018-08-18 01:20:10,426 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 01:20:10,428 INFO sqlalchemy.engine.base.Engine COMMIT


**Classical Mappings**  
The Declarative system, though highly recommended, is not required in order to use SQLAlchemy’s ORM.  
Outside of Declarative, any plain Python class can be mapped to any Table using the `mapper()` function directly; this less common usage is described at [Classical Mappings](http://docs.sqlalchemy.org/en/latest/orm/mapping_styles.html#classical-mapping).


## [Create an Instance of the Mapped Class](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#create-an-instance-of-the-mapped-class)

With mapping complete, we can create and inspect a User object:

In [8]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword'); ed_user.name

'ed'

In [9]:
ed_user.password

'edspassword'

In [10]:
print ed_user.id

None


In [11]:
str(ed_user.id)

'None'

Even though we didn’t specify it in the constructor, the `id` attribute still produces a value of None when we access it (as opposed to Python’s usual behavior of raising an `AttributeError` for an undefined attribute).  
SQLAlchemy’s instrumentation normally produces this default value for column-mapped attributes when first accessed.  
For those attributes where we’ve actually assigned a value, the instrumentation system is tracking those assignments for use within an eventual INSERT statement to be emitted to the database.

## [Creating a Session](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#creating-a-session)

We're ready to talk to our database.  
The ORM’s “handle” to the database is the `Session`.  
When we first set up the application, at the same level as our `create_engine()` statement, we define a Session class which will serve as a factory for new Session objects:

In [12]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine); Session

sessionmaker(class_='Session', autoflush=True, bind=Engine(sqlite:///:memory:), autocommit=False, expire_on_commit=True)

If you haven't defined an Engine yet, you can set it up like so:

In [13]:
Session = sessionmaker(); Session

sessionmaker(class_='Session', autoflush=True, bind=None, autocommit=False, expire_on_commit=True)

Now when you call `create_engine()` you can connect it to the Session using `configure()`:

In [14]:
Session.configure(bind=engine); Session

sessionmaker(class_='Session', autoflush=True, bind=Engine(sqlite:///:memory:), autocommit=False, expire_on_commit=True)

This custom-made Session class will create new Session objects which are bound to our database.  
Other transactional characteristics may be defined when calling sessionmaker as well; these are described in a later chapter.  
Then, whenever you need to have a conversation with the database, you instantiate a Session:

In [15]:
session = Session()
session

The above Session is associated with our SQLite-enabled Engine, but it hasn’t opened any connections yet.  
When it’s first used, it retrieves a connection from a pool of connections maintained by the Engine, and holds onto it until we commit all changes and/or close the session object.

## [Adding and Updating Objects](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#adding-and-updating-objects)

To persist our `User` object, we `add()` it to the `Session`:

In [16]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

At this point, we say that the instance is **pending**; no SQL has yet been issued and the object is not yet represented by a row in the database.  
The Session will issue the SQL to persist `Ed Jones` as soon as is needed, using a process known as a **flush**.  
If we query the database for Ed Jones, all pending information will first be flushed, and the query is issued immediately thereafter.  
For example, below we create a new Query object which loads instances of User.  
We `filter_by()` the name attribute of `ed`, and indicate that we’d like only the first result in the full list of rows.  
A User instance is returned which is equivalent to that which we’ve added:

In [17]:
our_user = session.query(User).filter_by(name='ed').first(); our_user

2018-08-18 01:20:10,565 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 01:20:10,567 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-08-18 01:20:10,569 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2018-08-18 01:20:10,572 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2018-08-18 01:20:10,573 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='edspassword')>

In fact, the Session has identified that the row returned is the same row as one already represented within its internal map of objects, so we actually got back the identical instance that was just added:

In [18]:
ed_user is our_user

True

The ORM concept at work here is known as an identity map and ensures that all operations upon a particular row within a Session operate upon the same set of data.  
Once an object with a particular primary key is present in the Session, all SQL queries on that Session will always return the same Python object for that particular primary key.    
Also an error will occur if an attempt is made to place a second, already-persisted object with the same primary key within the session.  
We can add more User objects all at once using the `add_all()` method:

In [19]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

We also need to update Ed's password because it's insecure:

In [20]:
ed_user.password = 'f8s7ccs'

The Session is paying attention.  
It knows, for example, that `Ed Jones` has been modified.

In [21]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

The Session also knows that three new User objects are pending:

In [22]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])

We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout.  
We do this via `commit()`.  
The Session emits the UPDATE statement for the password change on "ed", as well as INSERT statements for the three new User objects we’ve added:

In [23]:
session.commit()

2018-08-18 01:20:10,649 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2018-08-18 01:20:10,650 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 1)
2018-08-18 01:20:10,652 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-08-18 01:20:10,654 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2018-08-18 01:20:10,656 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-08-18 01:20:10,658 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2018-08-18 01:20:10,659 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-08-18 01:20:10,661 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2018-08-18 01:20:10,662 INFO sqlalchemy.engine.base.Engine COMMIT


The `commit()` flushes the remaining changes to the database, and commits the transaction.  
The connection resources referenced by the session are now returned to the connection pool.  
Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed.  
If we look at Ed’s id attribute, which earlier was None, it now has a value:

In [24]:
ed_user.id

2018-08-18 01:20:10,674 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 01:20:10,676 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2018-08-18 01:20:10,678 INFO sqlalchemy.engine.base.Engine (1,)


1

After the Session inserts new rows in the database, all newly generated identifiers and database-generated defaults become available on the instance, either immediately or via load-on-first-access.  
In this case, the entire row was re-loaded on access because a new transaction was begun after we issued `commit()`. SQLAlchemy by default refreshes data from a previous transaction the first time it’s accessed within a new transaction, so that the most recent state is available.  
The level of reloading is configurable as is described in [Using the Session](http://docs.sqlalchemy.org/en/latest/orm/session.html).

**Note About Session Object States**  
As our User object moved from being outside the Session, to inside the Session without a primary key, to actually being inserted, it moved between three out of four available “object states” - *transient*, *pending*, and *persistent*.  
Being aware of these states and what they mean is always a good idea - be sure to read [Quickie Intro to Object States](http://docs.sqlalchemy.org/en/latest/orm/session_state_management.html#session-object-states) for a quick overview.

## [Rolling Back](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#rolling-back)

The Session works within a transaction, so we can roll back changes that were made.  
Let's change `ed_user`'s user name.

In [25]:
ed_user.name = 'Edwardo'; ed_user

<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>

We can add another make-believe user, too:

In [26]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

When we query the session, we can see that they have been flushed into the current transaction.

In [27]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2018-08-18 01:26:29,734 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2018-08-18 01:26:29,736 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2018-08-18 01:26:29,737 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-08-18 01:26:29,739 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2018-08-18 01:26:29,741 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2018-08-18 01:26:29,742 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

Now let's `rollback()` the Session:

In [28]:
session.rollback()

2018-08-18 01:27:33,617 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [29]:
ed_user.name

2018-08-18 01:27:43,577 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-08-18 01:27:43,579 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2018-08-18 01:27:43,581 INFO sqlalchemy.engine.base.Engine (1,)


u'ed'

In [30]:
fake_user in session

False

Issuing a SQL SELECT query illustrates the changes made to the database.

In [32]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2018-08-18 01:30:44,084 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2018-08-18 01:30:44,086 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

## [Querying](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#querying)